In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
from glob import glob

In [2]:
IMAGE_SIZE = [224, 224]

train_path = 'train'
valid_path = 'test'

In [3]:
# add preprocessing layer to the front of VGG

pretrained_model = EfficientNetB0(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in pretrained_model.layers:
    layer.trainable = False
  
pretrained_model.layers[-1].trainable = True
pretrained_model.layers[-2].trainable = True
pretrained_model.layers[-3].trainable = True


In [4]:
# our layers - you can add more if you want
x = Flatten()(pretrained_model.output)
x2 = Dense(256, activation='relu')(x)
x3 = Dropout(0.2)(x2)
prediction = Dense(5, activation='softmax')(x3)

# create a model object
model = Model(inputs=pretrained_model.input, outputs=prediction)


# view the structure of the model
model.summary()

optimizer = Adam(lr = 0.0003)

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizer,
  metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [5]:
train_datagen = ImageDataGenerator(
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (IMAGE_SIZE),
                                                 batch_size = 32,
                                                 shuffle = True,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (IMAGE_SIZE),
                                            batch_size = 32,
                                            shuffle = True,
                                            class_mode = 'categorical')

Found 25102 images belonging to 5 classes.
Found 6236 images belonging to 5 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
785/785 [==============================] - 408s 507ms/step - loss: 1.4800 - accuracy: 0.4323 - val_loss: 1.1145 - val_accuracy: 0.5520
Epoch 2/5
785/785 [==============================] - 392s 500ms/step - loss: 1.1037 - accuracy: 0.5606 - val_loss: 1.0764 - val_accuracy: 0.5789
Epoch 3/5
785/785 [==============================] - 392s 499ms/step - loss: 1.0204 - accuracy: 0.5992 - val_loss: 1.0620 - val_accuracy: 0.5868
Epoch 4/5
785/785 [==============================] - 391s 498ms/step - loss: 0.9476 - accuracy: 0.6348 - val_loss: 1.0339 - val_accuracy: 0.5961
Epoch 5/5
785/785 [==============================] - 391s 498ms/step - loss: 0.8854 - accuracy: 0.6496 - val_loss: 1.0314 - val_accuracy: 0.5978


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")